In [17]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [18]:
input_image=tf.placeholder(tf.float32,[None,32,32,3])
label_y=tf.placeholder(tf.float32,[None,10])


def get_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def get_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv_2d(x,W):
    return tf.nn.conv2d(input=x,filter=W,strides=[1,1,1,1],padding="SAME")

def max_pool_3x3(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,2,2,1],padding="SAME")

In [19]:
#"the first layer filers shape [5,5,3,64] using 5X5X3 totals 64 filters"
W1=get_weights([5,5,3,64])
B1=get_bias([64])
H1=tf.nn.relu(conv_2d(input_image,W1)+B1)
MaxPool1=max_pool_3x3(H1)

#"the second layer filers shape [5,5,64,64] using 5X5X64 totals 64 filters"
W2=get_weights([5,5,64,64])
B2=get_bias([64])
H2=tf.nn.relu(conv_2d(MaxPool1,W2)+B2)
MaxPool2=max_pool_3x3(H2)
print(MaxPool2.shape)
#"the three layer"
flattern_MaPool2=tf.reshape(MaxPool2,[1,8*8*64])
W3=get_weights([8*8*64,120])
B3=get_bias([120])
FC1=tf.nn.relu(tf.matmul(flattern_MaPool2,W3)+B3)

W4=get_weights([120,84])
B4=get_bias([84])
FC2=tf.nn.relu(tf.matmul(FC1,W4)+B4)

W5=get_weights([84,10])
B5=get_bias([10])
pred_yhat=tf.nn.softmax(tf.matmul(FC2,W5)+B5)

(?, 8, 8, 64)


In [20]:
cost=tf.nn.softmax_cross_entropy_with_logits(labels=label_y,logits=pred_yhat)
train_op=tf.train.AdamOptimizer(1e-3).minimize(cost)
correct_prediction=tf.equal(tf.argmax(label_y,1),tf.argmax(pred_yhat,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [21]:
def read_decode(filename):
    filename_queue=tf.train.string_input_producer([filename],shuffle=True)#创建一个文件名队列（打乱顺序）
    reader=tf.TFRecordReader()
    _,serialized_example=reader.read(filename_queue)##读取队列数据
    features=tf.parse_single_example(serialized_example,
                                             features={
                                                 "label":tf.FixedLenFeature([],tf.int64),
                                                 "image":tf.FixedLenFeature([],tf.string)
                                             })
    image=tf.decode_raw(features["image"],tf.uint8)##重新编码成图片
    image=tf.reshape(image,[32,32,3])
    image=tf.cast(image,tf.int64)
    label=tf.cast(features['label'],tf.int32)
    print(image,label)
    return image,label

In [22]:
def get_batch_cifar10(filename,batchsize):
#     file_name_queue=tf.train.string_input_producer([filename],shuffle=True)
    image,label=read_decode(filename)
    
    min_after_dequeue=1000
    capacity=min_after_dequeue+3*batchsize
    
    image_batch,label_batch=tf.train.shuffle_batch([image,label],batch_size=batchsize,
                                                   capacity=capacity,min_after_dequeue=min_after_dequeue)
    return image_batch,label_batch

In [23]:
filename="cifar10_batch1_train.tfrecords"
raw_image,label=read_decode(filename)

Tensor("Cast_11:0", shape=(32, 32, 3), dtype=int64) Tensor("Cast_12:0", shape=(), dtype=int32)


In [24]:
init=tf.global_variables_initializer()
image_batch,label_batch=get_batch_cifar10("./cifar10_batch1_train.tfrecords",batchsize=1)
with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()###创建一个协调器，管理线程
    threads=tf.train.start_queue_runners(sess,coord)#启动队列
    
    for i in range(1000+1):
        batch_x,batch_y=sess.run([image_batch,label_batch])
#         print(batch_x.shape,sess.run(batch_y))
#         print(sess.run(batch_y))
        batch_y=sess.run(tf.one_hot(batch_y,10,1,0))
#         print(batch_x.shape,batch_y)
        if i%100==0:
            train_acc=sess.run(accuracy,feed_dict={input_image:batch_x,label_y:batch_y})
            print("train_step {} train_acc {}".format(i,train_acc))
        sess.run(train_op,feed_dict={input_image:batch_x,label_y:batch_y})
#     print("test accuracy {}".format(sess.run(accuracy,feed_dict={input_image:mnist.test.images,label_y:mnist.test.labels})))

Tensor("Cast_13:0", shape=(32, 32, 3), dtype=int64) Tensor("Cast_14:0", shape=(), dtype=int32)
train_step 0 train_acc 0.0
train_step 100 train_acc 0.0
train_step 200 train_acc 0.0
train_step 300 train_acc 0.0
train_step 400 train_acc 0.0
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[{{node input_producer_1/input_producer_1_EnqueueMany}} = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer_1, input_producer_1/Const, ^input_producer_1/Assert/Assert)]]


KeyboardInterrupt: 